In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn import model_selection

In [ ]:
from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT)

In [ ]:
PROJ = "My Drive/Colab Notebooks/AML Workspace/EM_KDE_optimized" # This is a custom path.
PROJECT_PATH = join(ROOT, PROJ)

In [ ]:
from importlib.machinery import SourceFileLoader
utils = SourceFileLoader('utils', join(PROJECT_PATH, 'utils.py')).load_module()
plot = SourceFileLoader('plot', join(PROJECT_PATH, 'plot.py')).load_module()

In [ ]:
from utils import e_step, m_step, calculate_log_likelihood, is_converged
from plot import plot_kde

 Load data

In [ ]:
data = loadmat(join(ROOT, 'My Drive/Colab Notebooks/AML Workspace/faithfull/faithful.mat'))['X']

 Real world data (may make sense to crop end, since it's quite big)

In [ ]:
# data = np.genfromtxt(join(ROOT, 'My Drive/Colab Notebooks/AML Workspace/data/winequality-white.csv'), delimiter=';')[1:,:80]

 Testing with higher dimension data<br>

In [ ]:
# np.random.shuffle(data)
# data = np.concatenate([data, loadmat(join(ROOT, 'My Drive/Colab Notebooks/AML Workspace/faithfull/faithful.mat'))['X']], axis=1)

In [ ]:
# data = data[:250]  # taking only a small part for testing

In [ ]:
num_data, dim = data.shape

K-fold crossvalidation

In [ ]:
K = 250
CV = model_selection.KFold(n_splits=K, shuffle=False)

 Loop until you're happy

In [ ]:
epsilon = 1e-3
sigma = np.eye(dim)
log_likelihood = np.asarray([])
i = 0
while True:
    i += 1
    sigmas = []
    R = np.linalg.cholesky(sigma)
    A = data.dot(np.linalg.inv(R).T)
    for train_index, test_index in CV.split(A):
        # extract training and test set for current CV fold
        a_test = A[test_index, :]
        a_train = A[train_index, :]
        x_test = data[test_index, :]
        x_train = data[train_index, :]

        # E step
        responsibility = e_step(a_test, a_train, R)

        # M step
        sigmas.append(m_step(x_test, x_train, responsibility))
    sigma = np.array(sigmas).sum(axis=1).mean(axis=0)
    R = np.linalg.cholesky(sigma)
    A = data.dot(np.linalg.inv(R).T)
    _log_likelihood = []
    for train_index, test_index in CV.split(A):
        # extract training and test set for current CV fold
        x_train = A[train_index, :]
        x_test = A[test_index, :]
        _log_likelihood.append(calculate_log_likelihood(x_test, x_train, R))
    log_likelihood = np.append(log_likelihood, np.asarray(_log_likelihood).mean())
    if is_converged(log_likelihood, epsilon):
        break

In [ ]:
plt.figure(1)
plt.plot(log_likelihood)
plt.xlabel('Iterations')
plt.ylabel('Log-likelihood')
plt.show()

In [ ]:
plot_kde(data, sigma, 0.1)